# Building a simple search index

_(Inspired by and borrowed heavily from: Collective Intelligence - [Luís F. Simões](mailto:luis.simoes@vu.nl). IR version and assignments by J.E. Hoeksema, 2014-11-03. Converted to Python 3 and minor changes by Tobias Kuhn, 2015-10-22.)_

*******

**THIS IS NOT THE FINAL VERSION OF THE DOCUMENT. PLEASE UPDATE TO THE LATEST VERSION BEFORE DOING THE ASSIGNMENTS AT THE BOTTOM.**

This notebook's purpose is to build a simple search index (to be used for boolean retrieval)

## Loading the data

In [1]:
Summaries_file = 'data/air__Summaries.pkl.bz2'
Abstracts_file = 'data/air__Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at how the data looks for our example paper:

In [3]:
Summaries[26488732]

paper(title='Use of Whole-Genome Sequencing to Link Burkholderia pseudomallei from Air Sampling to Mediastinal Melioidosis, Australia.', authors=['Currie BJ', 'Price EP', 'Mayo M', 'Kaestli M', 'Theobald V', 'Harrington I', 'Harrington G', 'Sarovich DS'], year=2015, doi='10.3201/eid2111.141802')

In [4]:
Abstracts[26488732]

'The frequency with which melioidosis results from inhalation rather than percutaneous inoculation or ingestion is unknown. We recovered Burkholderia pseudomallei from air samples at the residence of a patient with presumptive inhalational melioidosis and used whole-genome sequencing to link the environmental bacteria to B. pseudomallei recovered from the patient.'

## Some utility functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive - you may have to make them smarter in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long titles or author lists, and links to the paper's  DOI (when available).
    """
    s = Summaries[ id ]
    
    title = ( s.title if s.title[-1]!='.' else s.title[:-1] )
    title = title[:150].rstrip() + ('' if len(title)<=150 else '...')
    if s.doi!='':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    
    authors = ', '.join( s.authors[:5] ) + ('' if len(s.authors)<=5 else ', ...')
    
    lines = [
        title,
        authors,
        str(s.year),
        '<small>id: %d%s</small>' % (id, extra_text)
        ]
    
    display( HTML( '<blockquote>%s</blockquote>' % '<br>'.join(lines) ) )
    
def display_abstract( id, highlights=[]):
    """
    Function for displaying an abstract. Includes optional (naive) highlighting
    """
    a = Abstracts[ id ]
    for h in highlights:
        a = re.sub(r'\b(%s)\b'%h,'<mark>\\1</mark>',a, flags=re.IGNORECASE)
    display( HTML( '<blockquote>%s</blockquote' % a ) )
    
display_summary(26488732)
display_abstract(26488732, ['sequencing'])

## Creating our first index

We will now create an [Inverted Index](http://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html) based on the words in the abstracts of the papers in our dataset. We will once again use our *defaultdict with a default value of an empty set* trick to ensure a document is only added to a posting list once.

Our end result will be a dictionary, where each key is a term, and each value is a posting list, represented by a `set` of paper IDs.

Note that not every paper in our summaries set has an abstract; we will only index papers for which an abstract is present.

In [7]:
from collections import defaultdict

inverted_index = defaultdict(set)

# Takes a while
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

In [8]:
print(inverted_index['sequencing'])

{26431488, 18087937, 18044421, 22678023, 21038088, 17596935, 10852872, 24140300, 24039948, 22278670, 19593230, 21745680, 26157585, 19901455, 9361427, 21395476, 15653909, 18297876, 9570839, 12114969, 11724825, 26412057, 9180697, 23642653, 12171293, 25880607, 4013081, 24083487, 19581474, 18521631, 21468708, 24551973, 21936679, 15303720, 15258153, 24507435, 15943213, 20519470, 25348656, 25109864, 11981877, 21459509, 23698488, 20955193, 9504827, 18430012, 24446525, 24072765, 16436289, 16989772, 23289422, 22038094, 24823374, 26304593, 25877070, 23646291, 24456276, 23275093, 23851093, 18024023, 23880792, 9694291, 17365082, 24719455, 16972383, 14525537, 10747999, 7781991, 2007144, 22993513, 25710698, 15112299, 19700844, 26414193, 9582197, 14717560, 25463417, 24056441, 24739965, 22835837, 25482367, 16408190, 11334785, 25853058, 22624387, 23586944, 11683465, 12573836, 23461517, 18802318, 22771343, 17229452, 16962705, 16722067, 8118931, 2196119, 25405592, 8217167, 16425634, 25047205, 25831590, 2

We can now use this inverted index to answer simple one-word queries, for example to get an arbitrary paper that contains the word 'sequencing':

In [9]:
query_word = 'sequencing'
first_paper = list(inverted_index[query_word])[0] # Note that we convert a set into a list in order to fetch its first element
display_summary(first_paper) 
display_abstract(first_paper,[query_word])

## Assignments

- Construct two functions (`or_query` and `and_query`) that will each take as input a single string, consisting of one or more words, and return a list of matching documents. `or_query` will return documents that contain at least one of the words in the query, while `and_query` requires all query terms to be present in the documents.
  
_Note that you can use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query. You can also exploit the fact that the posting lists are [sets](https://docs.python.org/2/library/stdtypes.html#set), which means you can easily perform set operations such as union, difference and intersect on them._

- How many hits does `or_query('The Who')` return? Given the nature of our dataset, how many documents do you think are actually about [The Who](http://en.wikipedia.org/wiki/The_Who)? What could you do to prevent these kind of incorrect results? *(Note that you do not have to implement this yet)*

- Why does `and_query('Evolutionary Process')` not return our example paper 23144668, while it does speak about evolutionary processes in the abstract? *(Note that you do not have to implement anything to fix this yet)*